# Batch Inference with SageMaker Batch Transform

This notebook demonstrates how to create an Amazon SageMaker Batch Transform Job by using the last approved model taken from the Amazon SageMaker Model Registry.

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Get latest approved model from the Amazon SageMaker Model Registry
 - Run a Batch Inference Pipeline

***

# Step 1 - Import Modules

Here we’ll import some libraries and define some variables.

In [ ]:
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import os
import sagemaker.session
from sagemaker.pytorch import PyTorchModel
import traceback

In [ ]:
s3_client = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")

In [ ]:
BASE_DIR = os.getcwd()

BASE_DIR

Create a SageMaker Session and save the default region and the execution role in some Python variables

In [ ]:
sagemaker_session = sagemaker.Session()
region = boto3.session.Session().region_name
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()

***

# Step 2 - Get last approved model from Model Registry

Let's retrieve the model information stored in the Amazon SageMaker Model Registry

In [ ]:
model_package_group_name = "mlops-demo-batch-p-ypcnnfv6l5mn"

Get the last approved model from the Model Package Group defined

In [ ]:
try:
    # Get the latest approved model package
    response = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        ModelApprovalStatus="Approved",
        SortBy="CreationTime",
        SortOrder="Descending",
        MaxResults=1,
    )
    approved_packages = response["ModelPackageSummaryList"]

    # Return error if no packages found
    if len(approved_packages) == 0:
        error_message = ("No approved ModelPackage found for ModelPackageGroup: {}".format(model_package_group_name))
        print("{}".format(error_message))

        raise Exception(error_message)

    model_package = approved_packages[0]
    print("Identified the latest approved model package: {}".format(model_package))
except ClientError as e:
    stacktrace = traceback.format_exc()
    error_message = e.response["Error"]["Message"]
    print("{}".format(stacktrace))

    raise Exception(error_message)

Retrieve the model information by describing the package

In [ ]:
model_package_arn = model_package["ModelPackageArn"]

model_package_arn

In the previous notebook [01-ML-Model-Train](./../01-model-build/01-ML-Model-Train.ipynb) we have stored the training job name in the Model Package Group Description. Let's get this information for the model card lineage in the Model Card section

In [ ]:
try:
    model_package = sagemaker_client.describe_model_package(
        ModelPackageName=model_package_arn
    )

    print("{}".format(model_package))

    if len(model_package) == 0:
        error_message = ("No ModelPackage found for: {}".format(model_package_arn))
        print("{}".format(error_message))

        raise Exception(error_message)
except ClientError as e:
    stacktrace = traceback.format_exc()
    error_message = e.response["Error"]["Message"]
    print("{}".format(stacktrace))

    raise Exception(error_message)

In [ ]:
! chmod +x buildspec.sh
! ./buildspec.sh pipelines/batch_inference/inference

In [ ]:
! aws s3 cp ./pipelines/batch_inference/dist/inference/sourcedir.tar.gz s3://{default_bucket}/artifacts/inference/sourcedir.tar.gz

### Create SageMaker model

This method can be used for creating a SageMaker model

In [ ]:
inference_framework_version = "1.12"
inference_python_version = "py38"
inference_instance_type = "ml.m5.large"

In [ ]:
model = PyTorchModel(
        entry_point="inference.py",
        name=model_package_group_name + "-" + str(model_package["ModelPackageVersion"]),
        framework_version=str(inference_framework_version),
        py_version=inference_python_version,
        source_dir="s3://{}/artifacts/inference/sourcedir.tar.gz".format(default_bucket),
        model_data=model_package["InferenceSpecification"]["Containers"][0]["ModelDataUrl"],
        role=role,
        sagemaker_session=sagemaker_session
    )

In [ ]:
model.create(
    instance_type=inference_instance_type
)

***

### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [ ]:
from pipelines.batch_inference.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    model_name=model_package_group_name + "-" + str(model_package["ModelPackageVersion"]),
    role=role,
    default_bucket=default_bucket
)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [ ]:
execution = pipeline.start(
    parameters = {
        "InputPath":"s3://{}/e2e-base/data/inference/input".format(default_bucket),
        "OutputPath":"s3://{}/e2e-base/data/inference/output".format(default_bucket)
    }
)

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
execution.describe()

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()